In [1]:
import numpy as np
import pandas as pd

In [2]:
from fuzzywuzzy import fuzz, process
from tqdm import tqdm

In [7]:
pd.options.display.max_colwidth = 1000

In [3]:
# Load policy titles from file
with open('../data/Overton/covid_03_01/combined_policy_titles.txt') as f:
    unique_policy_titles = f.read().splitlines()
len(unique_policy_titles)

189

In [4]:
parliament = pd.read_csv('../data/parliament_argus.csv')
parliament.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61296 entries, 0 to 61295
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     61296 non-null  object
 1   content  61295 non-null  object
 2   speaker  61296 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [10]:
cabinet = pd.read_csv('../data/cabinet/pdf_files.csv')
cabinet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      108 non-null    int64 
 1   file_name       108 non-null    object
 2   url             108 non-null    object
 3   published_date  108 non-null    object
 4   file_type       108 non-null    object
 5   file_content    106 non-null    object
dtypes: int64(1), object(5)
memory usage: 5.2+ KB


In [16]:
## Matching the Overton policy titles with parliament discussion content
# drop rows with missing values in 'content' column
parliament = parliament.dropna(subset=['content'])

# Create a regular expression pattern
pattern = '|'.join(unique_policy_titles)
pattern = pattern.lower().strip()

# Filter rows that contain any policy title
matched_rows = parliament[parliament['content'].str.lower().str.contains(pattern, regex=True)]

matched_rows

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.


,date,content,speaker
7697,2020-12-03,"Child poverty is another. We took some steps in the last Parliament by legislating the child poverty reduction legislation and started to take that seriously, starting to actually shine a light on what is happening with child poverty in New Zealand. I'm confident, with the leadership of the Prime Minister and also the Minister for Children, the Hon Kelvin Davis, that we will see some really significant change this term, because we have to. We have to make gains and improve the situation of those living in depravation and poverty. Children don't ask and don't choose to live in poverty; it's a circumstance put upon them, and we need to support them, their whānau, their families, their parents to make a difference for them",Hon ANDREW LITTLE (Minister of Health)
28228,2021-04-08,"Next week, the annual review debate will begin. The debate is the committee stage of the Appropriation (2019/20 Confirmation and Validation) Bill and is 10 hours long. It will be conducted under the revised procedures for committee stage, with Ministers responding to questions from members. On Tuesday, the House will consider a motion to approve orders made under the COVID-19 Public Health Response Act 2020 and a motion to continue the Act until the end of the year. Wednesday will be a members' day. A debate on the report of the Finance and Expenditure Committee on the Budget Policy Statement 2021 will replace the general debate. The second reading of the Fair Trading Amendment Bill will also be considered next week",Hon GRANT ROBERTSON (Deputy Prime Minister) Leader of the House


In [12]:
# Load testing policy titles from file
with open('../data/Overton/covid_03_01/overton_policy_titles1.txt') as f:
    unique_policy_titles1 = f.read().splitlines()
len(unique_policy_titles1)

125

In [5]:
# Matching the Overton policy titles with parliament discussion content
# Drop rows with missing values in 'content' column
parliament = parliament.dropna(subset=['content'])

# Create a regular expression pattern
pattern = '|'.join(unique_policy_titles)
pattern = pattern.lower().strip() # convert to lowercase and remove spaces before/after

# Filter rows that contain any policy title
matched_rows = parliament[parliament['content'].str.lower().str.contains(pattern, regex=True)]

In [7]:
# Extract matching policy titles
matched_rows['matching_title'] = matched_rows['content'].str.lower().str.extract(f"({pattern})")
# Reset the index of matched_rows
matched_rows

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,content,speaker,matching_title
1090,2020-06-24,"Fluffy"", as one of my colleagues says. They want to see that the Government is focused on them. They want to know the Government is focused on them and improving their quality of life. So yep, Treasury's Living Standards Framework—no disagreement around how important that is. It was created under the Rt Hon Bill English's leadership. There's no argument about that it is the Government's job to improve the quality of people's lives and their standard of living and their wellbeing. To actually, though, put it in the Public Finance Act, and then expect economic analysists at Treasury to measure it and report on it are two very, very different things",Hon LOUISE UPSTON,living standards framework
2031,2020-06-24,"100 million is being invested through Budget 2020 and the COVID recovery package to provide people with jobs, to ensure that people who are guiding in Franz Josef—whose jobs have gone, through no fault of their own, through the redundancies that have been associated with the closure of our borders to protect our country and the drop-off in tourism—can work. They've moved from Franz to Springfield, Arthur's Pass and are working now on protecting the landscapes, protecting our tussock grasslands. It is a sign of this Government's commitment to investing in people, investing in Papatūānuku, so that we have a healthy future",Hon EUGENIE SAGE (Minister of Conservation),budget 2020
2111,2020-05-26,"833 million in Budget 2020 for disability support services. There's never been such an investment in disability support services. In our first year, when we came in, we invested $211 million. Last year, in our Wellbeing Budget, we invested $348 million for disability support services. But this year, this huge investment of $833 million is historic. We know that one of the things that the disability support services sector usually does is, after every year, it actually gets a top-up. So this huge investment of $833 million should ensure that we won't actually need such a huge top-up in the future. This year, we are topping up by just over $100 million, but as I say, one of the reasons why we're investing that much is to ensure that in the future we won't have to come back and top it up",Hon JENNY SALESA,budget 2020
2462,2020-06-02,"A further $10.2 million is being provided to improve the Defence Force's internal security capability. Budget 2020 also contains contingent funding to support the delivery of the Defence Capability Plan 2019—once again, a very forward-reaching capability plan, built on the back of the coalition agreement, whereby in this coalition Government, Labour agreed that we would review the last Government's capability plan and we'd put some oomph into it. So we are looking forward down the line to seeing this country own its very first landing platform dock, which will be a heck of an asset when it comes to dealing with humanitarian disaster and relief operations, something that we're seeing more and more of as the climate has changed. The Defence Capability Plan—I covered that. Moving on",Hon RON MARK (Minister of Defence),budget 2020
3367,2020-03-11,"Also within this work that we have undertaken in the Wellbeing Budget 2020, we are looking at ""Just Transition—Supporting New Zealand in the transition to a climate-resilient, sustainable, and low-emission economy"". New Zealand First has always said—and we've worked with the farmers closely and come up with a good solution, despite the members opposite—that a good proposition for New Zealand exporters is to stand up in our world market and put the provenance to our buyers around the world and to have that story to tell about happy employees working and earning a good living in their job and about an economy that understands their world commitment to carbon emissions, so that our buyers around the world can take satisfaction and take pride and actually pay a premium for New Zealand products because they understand

In [3]:
matched_rows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      178 non-null    int64 
 1   date            178 non-null    object
 2   content         178 non-null    object
 3   speaker         178 non-null    object
 4   matching_title  178 non-null    object
dtypes: int64(1), object(4)
memory usage: 7.1+ KB


In [13]:
matched_rows.to_csv('../data/Overton/covid_03_01/matched_policy_titles_with_parliament.csv')

In [2]:
matched_rows = pd.read_csv('../data/Overton/covid_03_01/matched_policy_titles_with_parliament.csv')

In [4]:
import nltk
nltk.download('punkt')

def extract_context(title, content):
    sentences = nltk.sent_tokenize(content)
    index = [i for i, sentence in enumerate(sentences) if title in sentence]
    if index:
        index = index[0]
        start = max(0, index - 1)
        end = min(len(sentences), index + 2)
        return ' '.join(sentences[start:end])
    else:
        return ''

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
matched_rows['content'] = matched_rows['content'].str.lower()

In [11]:
matched_rows['matching_sentences'] = matched_rows.apply(lambda row: extract_context(row['matching_title'], row['content']), axis=1)

In [12]:
matched_rows.head()

,Unnamed: 0,date,content,speaker,matching_title,matching_sentences
0,1090,2020-06-24,"fluffy"", as one of my colleagues says. they want to see that the government is focused on them. they want to know the government is focused on them and improving their quality of life. so yep, treasury's living standards framework—no disagreement around how important that is. it was created under the rt hon bill english's leadership. there's no argument about that it is the government's job to improve the quality of people's lives and their standard of living and their wellbeing. to actually, though, put it in the public finance act, and then expect economic analysists at treasury to measure it and report on it are two very, very different things",Hon LOUISE UPSTON,living standards framework,"they want to know the government is focused on them and improving their quality of life. so yep, treasury's living standards framework—no disagreement around how important that is. it was created under the rt hon bill english's leadership."
1,2031,2020-06-24,"100 million is being invested through budget 2020 and the covid recovery package to provide people with jobs, to ensure that people who are guiding in franz josef—whose jobs have gone, through no fault of their own, through the redundancies that have been associated with the closure of our borders to protect our country and the drop-off in tourism—can work. they've moved from franz to springfield, arthur's pass and are working now on protecting the landscapes, protecting our tussock grasslands. it is a sign of this government's commitment to investing in people, investing in papatūānuku, so that we have a healthy future",Hon EUGENIE SAGE (Minister of Conservation),budget 2020,"100 million is being invested through budget 2020 and the covid recovery package to provide people with jobs, to ensure that people who are guiding in franz josef—whose jobs have gone, through no fault of their own, through the redundancies that have been associated with the closure of our borders to protect our country and the drop-off in tourism—can work. they've moved from franz to springfield, arthur's pass and are working now on protecting the landscapes, protecting our tussock grasslands."
2,2111,2020-05-26,"833 million in budget 2020 for disability support services. there's never been such an investment in disability support services. in our first year, when we came in, we invested $211 million. last year, in our wellbeing budget, we invested $348 million for disability support services. but this year, this huge investment of $833 million is historic. we know that one of the things that the disability support services sector usually does is, after every year, it actually gets a top-up. so this huge investment of $833 million should ensure that we won't actually need such a huge top-up in the future. this year, we are topping up by just over $100 million, but as i say, one of the reasons why we're investing that much is to ensure that in the future we won't have to come back and top it up",Hon JENNY SALESA,budget 2020,833 million in budget 2020 for disability support services. there's never been such an investment in disability support services.
3,2462,2020-06-02,"a further $10.2 million is being provided to improve the defence force's internal security capability. budget 2020 also contains contingent funding to support the delivery of the defence capability plan 2019—once again, a very forward-reaching capability plan, built on the back of the coalition agreement, whereby in this coalition government, labour agreed that we would review the last government's capability plan and we'd put some oomph into it. so we are looking forward down the line to seeing this country own its very first landing platform dock, which will be a heck of an asset when it comes to dealing with humanitarian disaster and relief operations, something that we're seeing more and more of as the climate has changed. the defence capability plan—i cov

In [25]:
unique_matching_titles = matched_rows['matching_title'].unique()
print(unique_matching_titles)

['living standards framework' 'budget 2020' 'he waka eke noa'
 'vote business, science and innovation'
 'four steps closer to sustainable tourism'
 'youth justice indicators summary report' 'response to he pou a rangi'
 'māori electoral option' 'strategic tourism assets protection'
 'whakamaua' 'action for healthy waterways'
 'digital technologies industry transformation plan'
 "new zealand's productivity performance"]


In [26]:
# create a dictionary with matching_title as key and original title as value
title_dict = {
    "living standards framework": "He Kāhui Waiora: Living Standards Framework and He Ara Waiora COVID-19: Impacts on Wellbeing (DP 20/02)",
    "budget 2020": "Key Documents - Budget 2020 Information Release",
    "he waka eke noa": "COVID-19 Economic Package at a Glance: He Waka Eke Noa: We Are All Working Together - April 2020",
    "vote business, science and innovation": "Vote Business, Science and Innovation - Economic Development and Infrastructure Sector - Estimates 2021/22",
    "four steps closer to sustainable tourism": "Not 100% – but four steps closer to sustainable tourism",
    "youth justice indicators summary report": "Youth Justice Indicators Summary Report December 2020",
    "response to he pou a rangi": "The Government’s response to He Pou a Rangi – Climate Change Commission’s recommendations",
    "māori electoral option": "Regulatory Impact Statement: Timing and frequency of the Māori Electoral Option",
    "strategic tourism assets protection": "Strategic Tourism Assets Protection Programme : Six-Month Update",
    "whakamaua": "Whakamaua: Māori Health Action Plan 2020-2025",
    "action for healthy waterways": "Action for healthy waterways: Section 32 evaluation report",
    "digital technologies industry transformation plan": "Releasing the draft Digital Technologies Industry Transformation Plan",
    "new zealand's productivity performance": "New Zealand's Productivity Performance: Taking a Broader View (AN 22/05)",
}

In [27]:
# create a new column with original titles based on matching_title
matched_rows['original_policy_title'] = matched_rows['matching_title'].map(title_dict)

In [51]:
# Load 163 overton policy titles from file
with open('../data/Overton/covid_03_01/overton_policy_titles.txt') as f:
    overton_policy_titles = f.read().splitlines()
len(overton_policy_titles)

163

In [60]:
matched_rows.head()

,Unnamed: 0,date,content,speaker,matching_title,matching_sentences,original_policy_title
0,1090,2020-06-24,"fluffy"", as one of my colleagues says. they want to see that the government is focused on them. they want to know the government is focused on them and improving their quality of life. so yep, treasury's living standards framework—no disagreement around how important that is. it was created under the rt hon bill english's leadership. there's no argument about that it is the government's job to improve the quality of people's lives and their standard of living and their wellbeing. to actually, though, put it in the public finance act, and then expect economic analysists at treasury to measure it and report on it are two very, very different things",Hon LOUISE UPSTON,living standards framework,"they want to know the government is focused on them and improving their quality of life. so yep, treasury's living standards framework—no disagreement around how important that is. it was created under the rt hon bill english's leadership.",He Kāhui Waiora: Living Standards Framework and He Ara Waiora COVID-19: Impacts on Wellbeing (DP 20/02)
1,2031,2020-06-24,"100 million is being invested through budget 2020 and the covid recovery package to provide people with jobs, to ensure that people who are guiding in franz josef—whose jobs have gone, through no fault of their own, through the redundancies that have been associated with the closure of our borders to protect our country and the drop-off in tourism—can work. they've moved from franz to springfield, arthur's pass and are working now on protecting the landscapes, protecting our tussock grasslands. it is a sign of this government's commitment to investing in people, investing in papatūānuku, so that we have a healthy future",Hon EUGENIE SAGE (Minister of Conservation),budget 2020,"100 million is being invested through budget 2020 and the covid recovery package to provide people with jobs, to ensure that people who are guiding in franz josef—whose jobs have gone, through no fault of their own, through the redundancies that have been associated with the closure of our borders to protect our country and the drop-off in tourism—can work. they've moved from franz to springfield, arthur's pass and are working now on protecting the landscapes, protecting our tussock grasslands.",Key Documents - Budget 2020 Information Release
2,2111,2020-05-26,"833 million in budget 2020 for disability support services. there's never been such an investment in disability support services. in our first year, when we came in, we invested $211 million. last year, in our wellbeing budget, we invested $348 million for disability support services. but this year, this huge investment of $833 million is historic. we know that one of the things that the disability support services sector usually does is, after every year, it actually gets a top-up. so this huge investment of $833 million should ensure that we won't actually need such a huge top-up in the future. this year, we are topping up by just over $100 million, but as i say, one of the reasons why we're investing that much is to ensure that in the future we won't have to come back and top it up",Hon JENNY SALESA,budget 2020,833 million in budget 2020 for disability support services. there's never been such an investment in disability support services.,Key Documents - Budget 2020 Information Release
3,2462,2020-06-02,"a further $10.2 million is being provided to improve the defence force's internal security capability. budget 2020 also contains contingent funding to support the delivery of the defence capability plan 2019—once again, a very forward-reaching capability plan, built on the back of the coalition agreement, whereby in this coalition government, labour agreed that we would review the last government's capability plan and we'd put some oomph into it. so we are looking forward down the line to seeing this country own its very first landing platfor

In [21]:
# Extract the 'speaker' and 'matching_title' columns
nodes = pd.concat([matched_rows['speaker'], matched_rows['matching_title']]).unique()

# Create a dictionary to map each node to a numeric id
node_dict = {node: i for i, node in enumerate(nodes)}

# Create a list of edges, where each edge is represented by a tuple of the source node and target node
edges = []
for i, row in matched_rows.iterrows():
    speakers = row['speaker']
    matching_title = row['matching_title']
    edges.append((speakers, matching_title))

# Create a dataframe from the list of edges
edges_df = pd.DataFrame(edges, columns=['Source', 'Target'])

# Replace the 'Source' and 'Target' columns with their corresponding numeric ids
edges_df['Source'] = edges_df['Source'].map(node_dict)
edges_df['Target'] = edges_df['Target'].map(node_dict)

# Map the numeric ids back to the original 'authors' and 'policy_doc_source' values
edges_df['Source'] = edges_df['Source'].map(dict(zip(range(len(node_dict)), node_dict)))
edges_df['Target'] = edges_df['Target'].map(dict(zip(range(len(node_dict)), node_dict)))

# Save the edges dataframe to a CSV file
edges_df.to_csv('../data/Overton/covid_03_01/parliament_policy_title_edges.csv', index=False)

In [12]:
# Extract matching policy titles
matched_cabinet_rows['cabinet_matching_title'] = matched_cabinet_rows['file_content'].str.lower().str.extract(f"({pattern1})")
# Reset the index of matched_rows
matched_cabinet_rows

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,file_name,url,published_date,file_type,file_content,cabinet_matching_title
34,34,ALC2-20802021-COVID-19-Response-Further-Auckland-Community-Cases.pdf,https://www.dpmc.govt.nz/sites/default/files/2023-01/ALC2-20802021-COVID-19-Response-Further-Auckland-Community-Cases.pdf,2021-08-20,Cabinet Paper and Minute,"SENSITIVEOffice of the Minister for COVID-19 ResponseCabinetCOVID-19 RESPONSE: FURTHER AUCKLAND COMMUNITY CASESdProposal1 This paper reviews the situation in response to further community COVID-19ecases identified in Auckland since Case A was identified on 17 August 2021. Ipropose keeping all of New Zealand at Alert Level 4 with another review onsMonday 23 August.aRelationship with Government priorities2 This paper concerns the Government’s response to COVID-19.eSummaryl3 Following the decision to move Auckland and Coromandel to Alert Level 4 untile11.59pm on 24 August and the rest of New Zealand to Alert Level 4 until11.59pm on 20 August 2021, a decision is required about the approach for theRrest of New Zealand for the next period.4 As at 9am on Friday 20 August, 30 community cases have been detected thatare linked to the Auckland outbreak. There are 28 cases within Auckland andytwo within Wellington. The index case of the Auckland community outbreakremains unknown and source inv...",budget 2020


In [61]:
# Save the matched rows to a CSV file
matched_rows.to_csv('../data/Overton/covid_03_01/title_matched_parliament.csv', index=False)

In [8]:
parliament_policy_connection = pd.read_csv('../data/Overton/covid_03_01/title_matched_parliament.csv')
parliament_policy_connection.drop(columns='Unnamed: 0', inplace=True)

In [9]:
parliament_policy_connection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   date                   178 non-null    object
 1   content                178 non-null    object
 2   speaker                178 non-null    object
 3   matching_title         178 non-null    object
 4   matching_sentences     178 non-null    object
 5   original_policy_title  178 non-null    object
dtypes: object(6)
memory usage: 8.5+ KB


In [11]:
parliament_policy_connection['original_policy_title'].nunique()

13

In [12]:
with open('../data/Overton/covid_03_01/parliament_matched_title.txt', 'r') as file:
    parliament_matched_title = [line.strip() for line in file.readlines()]
parliament_matched_title

['He Kāhui Waiora: Living Standards Framework and He Ara Waiora COVID-19: Impacts on Wellbeing (DP 20/02)',
 'Key Documents - Budget 2020 Information Release',
 'COVID-19 Economic Package at a Glance: He Waka Eke Noa: We Are All Working Together - April 2020',
 'Vote Business, Science and Innovation - Economic Development and Infrastructure Sector - Estimates 2021/22',
 'Not 100% – but four steps closer to sustainable tourism',
 'Youth Justice Indicators Summary Report December 2020',
 'The Government’s response to He Pou a Rangi – Climate Change Commission’s recommendations',
 'Regulatory Impact Statement: Timing and frequency of the Māori Electoral Option',
 'Strategic Tourism Assets Protection Programme : Six-Month Update',
 'Whakamaua: Māori Health Action Plan 2020-2025',
 'Action for healthy waterways: Section 32 evaluation report',
 'Releasing the draft Digital Technologies Industry Transformation Plan',
 "New Zealand's Productivity Performance: Taking a Broader View (AN 22/05)"]

In [14]:
# Convert parliament_matched_title to lowercase
parliament_matched_title_lower = [title.lower() for title in parliament_matched_title]

# Convert citing_content['policy_doc_title'] to lowercase and strip whitespace
parliament_policy_connection['original_policy_title'] = parliament_policy_connection['original_policy_title'].str.lower().str.strip()

# Match titles
matched_titles_13_parliament = parliament_policy_connection[parliament_policy_connection['original_policy_title'].isin(parliament_matched_title_lower)]['original_policy_title'].tolist()

# Find unmatched titles
unmatched_titles_13_parliament = [title for title in parliament_matched_title_lower if title not in matched_titles_13_parliament]

In [15]:
# Print the matched and unmatched titles
matched_titles_13_parliament = list(set(matched_titles_13_parliament))
print("Matched titles:")
print(matched_titles_13_parliament)
print("Unmatched titles:")
print(unmatched_titles_13_parliament)

Matched titles:
['key documents - budget 2020 information release', 'youth justice indicators summary report december 2020', 'releasing the draft digital technologies industry transformation plan', 'covid-19 economic package at a glance: he waka eke noa: we are all working together - april 2020', 'the government’s response to he pou a rangi – climate change commission’s recommendations', 'strategic tourism assets protection programme : six-month update', "new zealand's productivity performance: taking a broader view (an 22/05)", 'he kāhui waiora: living standards framework and he ara waiora covid-19: impacts on wellbeing (dp 20/02)', 'regulatory impact statement: timing and frequency of the māori electoral option', 'action for healthy waterways: section 32 evaluation report', 'not 100% – but four steps closer to sustainable tourism', 'whakamaua: māori health action plan 2020-2025', 'vote business, science and innovation - economic development and infrastructure sector - estimates 2021/

In [16]:
science_policy_connection = pd.read_csv('../data/Overton/covid_03_01/science_policy_connection.csv')

In [18]:
science_policy_connection.drop(columns='index', inplace=True)

In [19]:
science_policy_connection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103 entries, 0 to 1102
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   authors_institution     1099 non-null   object
 1   authors                 1099 non-null   object
 2   policy_document_id      1103 non-null   object
 3   overton_url             1103 non-null   object
 4   policy_doc_title        1103 non-null   object
 5   policy_doc_description  418 non-null    object
 6   published_date          1103 non-null   object
 7   policy_doc_area         1103 non-null   object
 8   policy_doc_source       1103 non-null   object
 9   citing_paper_text       1067 non-null   object
 10  citing_paper_doi        1103 non-null   object
dtypes: object(11)
memory usage: 94.9+ KB


In [20]:
# create a new column 'parliament_related_discussion' in policy_research_content
science_policy_connection['parliament_related_discussion'] = ''
science_policy_connection['parliament_related_speaker'] = ''
science_policy_connection['parliament_discussion_date'] = ''

In [28]:
# Iterate over parliament_policy_connection
for index, row in parliament_policy_connection.iterrows():
    # Extract the matching title from the matched rows dataframe
    matching_title = row['original_policy_title'].lower().strip()

    # Find all matching rows in policy_research_content based on the lowercased policy_doc_title
    matching_rows = science_policy_connection.loc[science_policy_connection['policy_doc_title'] == matching_title]

    # Iterate over all matching rows
    for matching_index in matching_rows.index:
        # Extract the matching sentences and speakers from the matched rows dataframe
        matching_sentences = row['matching_sentences']
        matching_speakers = row['speaker']
        matching_date = row['date']

        # Set the value of the 'parliament_related_discussion' and 'parliament_related_speaker' columns to the matching results
        science_policy_connection.at[matching_index, 'parliament_related_discussion'] = matching_sentences
        science_policy_connection.at[matching_index, 'parliament_related_speaker'] = matching_speakers
        science_policy_connection.at[matching_index, 'parliament_discussion_date'] = matching_date

In [29]:
science_policy_parliament_connection = science_policy_connection

In [30]:
science_policy_parliament_connection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103 entries, 0 to 1102
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   authors_institution            1099 non-null   object
 1   authors                        1099 non-null   object
 2   policy_document_id             1103 non-null   object
 3   overton_url                    1103 non-null   object
 4   policy_doc_title               1103 non-null   object
 5   policy_doc_description         418 non-null    object
 6   published_date                 1103 non-null   object
 7   policy_doc_area                1103 non-null   object
 8   policy_doc_source              1103 non-null   object
 9   citing_paper_text              1067 non-null   object
 10  citing_paper_doi               1103 non-null   object
 11  parliament_related_discussion  1103 non-null   object
 12  parliament_related_speaker     1103 non-null   object
 13  par

In [33]:
science_policy_parliament_connection.to_csv('../data/Overton/covid_03_01/science_policy_parliament_connection.csv',index=False)

In [31]:
has_parliament_connection = science_policy_parliament_connection[science_policy_parliament_connection['parliament_related_discussion'].str.len() > 1]
has_parliament_connection

,authors_institution,authors,policy_document_id,overton_url,policy_doc_title,policy_doc_description,published_date,policy_doc_area,policy_doc_source,citing_paper_text,citing_paper_doi,parliament_related_discussion,parliament_related_speaker,parliament_discussion_date
2,Motu Economic and Public Policy Research,Arthur Grimes,treasurynz-f80edb83327a37bc996c8034f492312a,http://app.overton.io/document.php?policy_docu...,new zealand's productivity performance: taking...,NaN,2022-09-28,"['economy, business and finance', 'science and...",New Zealand Treasury,"On this second point, Grimes and Hyland (2020)...",10.1111/sjpe.12237,at the heart of this is our drive for stronger...,Hon DAVID PARKER (Associate Minister of Finance),2019-08-27
10,Victoria University of Wellington,Arthur Grimes,treasurynz-f80edb83327a37bc996c8034f492312a,http://app.overton.io/document.php?policy_docu...,new zealand's productivity performance: taking...,NaN,2022-09-28,"['economy, business and finance', 'science and...",New Zealand Treasury,"On this second point, Grimes and Hyland (2020)...",10.1111/sjpe.12237,at the heart of this is our drive for stronger...,Hon DAVID PARKER (Associate Minister of Finance),2019-08-27
19,Motu Economic and Public Policy Research,Richard Fabling,treasurynz-f80edb83327a37bc996c8034f492312a,http://app.overton.io/document.php?policy_docu...,new zealand's productivity performance: taking...,NaN,2022-09-28,"['economy, business and finance', 'science and...",New Zealand Treasury,"For example, Fabling and Sanderson (2013) find...",10.1016/j.jinteco.2012.08.008,at the heart of this is our drive for stronger...,Hon DAVID PARKER (Associate Minister of Finance),2019-08-27
83,University of Chicago,Chad Syverson,treasurynz-f80edb83327a37bc996c8034f492312a,http://app.overton.io/document.php?policy_docu...,new zealand's productivity performance: taking...,NaN,2022-09-28,"['economy, business and finance', 'science and...",New Zealand Treasury,Fabling (2021) summarises several New Zealand ...,10.1257/jel.49.2.326,at the heart of this is our drive for stronger...,Hon DAVID PARKER (Associate Minister of Finance),2019-08-27
86,University of Waikato,Lynda Sanderson,treasurynz-f80edb83327a37bc996c8034f492312a,http://app.overton.io/document.php?policy_docu...,new zealand's productivity performance: taking...,NaN,2022-09-28,"['economy, business and finance', 'science and...",New Zealand Treasury,"For example, Fabling and Sanderson (2013) find...",10.1016/j.jinteco.2012.08.008,at the heart of this is our drive for stronger...,Hon DAVID PARKER (Associate Minister of Finance),2019-08-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,University of Turku,Mikko Niemelä,treasurynz-bc13ae5f63cb982554314cd66475a593,http://app.overton.io/document.php?policy_docu...,he kāhui waiora: living standards framework an...,The COVID‑19 pandemic is having major impacts ...,2020-07-07,"['science and technology', 'education', 'labou...",New Zealand Treasury,NaN,10.1111/roiw.12409,what we've got is wellbeing objectives being i...,Rt Hon DAVID CARTER,2020-06-24
1099,University of Turku,Esa Karonen,treasurynz-bc13ae5f63cb982554314cd66475a593,http://app.overton.io/document.php?policy_docu...,he kāhui waiora: living standards framework an...,The COVID‑19 pandemic is having major impacts ...,2020-07-07,"['science and technology', 'education', 'labou...",New Zealand Treasury,NaN,10.1111/roiw.12409,what we've got is wellbeing objectives being i...,Rt Hon DAVID CARTER,2020-06-24
1100,Johns Hopkins University,Linda Steffel Olson,treasurynz-bc13ae5f63cb982554314cd66475a593,http://app.overton.io/document.php?policy_docu...,he kāhui waiora: living standards framework an...,The COVID‑19 pandemic is having major impacts ...,2020-07-07,"['science and technology', 'education', 'labou...",New Zealand Treasury,NaN,10.1177/000312240707200202,what we've got is wellbeing objectives being i...,Rt Hon DAVID CARTER,2020-06-24
1101,Johns Hopkins University,Doris R. Entwisle,treasuryn

In [32]:
has_parliament_connection['policy_document_id'].nunique()

4